In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import zipfile
import io
from datetime import datetime
import pyquery
import thread

## Environment

In [3]:
from dwre_tools.env import get_default_project, get_project, load_config, get_environment

In [4]:
PROJECT = get_project("swk")
ENV = get_environment("dev01", PROJECT)

## Login Session

In [5]:
webdavsession = requests.session()
webdavsession.auth=(ENV["username"], ENV["password"],)
bmsession = requests.session()

In [6]:
bmsession.post("https://{}/on/demandware.store/Sites-Site/default/ViewApplication-ProcessLogin".format(ENV["server"]),
               data=dict(
                    LoginForm_Login=ENV["username"],
                    LoginForm_Password=ENV["password"],
                    LocaleID="",
                    LoginForm_RegistrationDomain="Sites",
                    login=""
                ))

<Response [200]>

## Utilities

In [7]:
def wait_for_import():
    response = bmsession.get("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Status".format(ENV["server"]))
    response_q = pyquery.PyQuery( response.content)
    log_link = response_q.find("a:contains('Site Import')").eq(0).attr("href")
    # TODO check if this matches the import we are targetting, not just the first
    finished = False
    while not finished: 
        log_response = bmsession.get(log_link)
        if "finished successfully" in log_response.content:
            finished = True
        else:
            thread.sleep(2)
            
def wait_for_export(session):
    response = session.get("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Status".format(ENV["server"]))
    response_q = pyquery.PyQuery( response.content)
    log_link = response_q.find("a:contains('Site Export')").eq(0).attr("href")
    # TODO check if this matches the import we are targetting, not just the first
    finished = False
    while not finished: 
        log_response = bmsession.get(log_link)
        if "finished successfully" in log_response.content:
            finished = True
        else:
            thread.sleep(2)
            
def download_zip(url):
    r = requests.get(url)
    zipfile = zipfile.ZipFile(BytesIO(r.content))
    return zipfile

## Bootstrap

In [8]:
TOOL_VERSION = "1"
BOOTSTRAP_META = """<?xml version="1.0" encoding="UTF-8"?>
<metadata xmlns="http://www.demandware.com/xml/impex/metadata/2006-10-31">
    <type-extension type-id="OrganizationPreferences">
        <custom-attribute-definitions>
            <attribute-definition attribute-id="dwreMigrateCurrentVersion">
                <display-name xml:lang="x-default">DWRE Migrate Current Version</display-name>
                <type>string</type>
                <mandatory-flag>false</mandatory-flag>
                <externally-managed-flag>false</externally-managed-flag>
                <min-length>0</min-length>
            </attribute-definition>
            <attribute-definition attribute-id="dwreMigrateToolVersion">
                <display-name xml:lang="x-default">DWRE Migrate Tool Version</display-name>
                <type>string</type>
                <mandatory-flag>false</mandatory-flag>
                <externally-managed-flag>false</externally-managed-flag>
                <min-length>0</min-length>
                <default-value>{0}</default-value>
            </attribute-definition>
        </custom-attribute-definitions>
        <group-definitions>
            <attribute-group group-id="dwreMigrate">
                <display-name xml:lang="x-default">DWREMigrate</display-name>
                <attribute attribute-id="dwreMigrateCurrentVersion"/>
                <attribute attribute-id="dwreMigrateToolVersion"/>
            </attribute-group>
        </group-definitions>
    </type-extension>
</metadata>
""".format(TOOL_VERSION)

In [9]:
PREFERENCES = """<?xml version="1.0" encoding="UTF-8"?>
<preferences xmlns="http://www.demandware.com/xml/impex/preferences/2007-03-31">
    <custom-preferences>
        <all-instances>
            <preference preference-id="dwreMigrateToolVersion">1</preference>
        </all-instances>
    </custom-preferences>
</preferences>
"""

VERSION = """###########################################
# Generated file, do not edit.
# Copyright (c) 2015 by Demandware, Inc.
###########################################
15.5.2
"""

In [10]:
dest_file = "DWREMigrateBootstrap_{}".format(datetime.now().isoformat())

In [11]:
bootstrap_package_file = io.BytesIO()
bootstrap_package_zip = zipfile.ZipFile(bootstrap_package_file, "w")

In [12]:
bootstrap_package_zip.writestr("{}/version.txt".format(dest_file), VERSION)
bootstrap_package_zip.writestr("{}/preferences.xml".format(dest_file), PREFERENCES)
bootstrap_package_zip.writestr("{}/meta/system-objecttype-extensions.xml".format(dest_file), BOOTSTRAP_META)
bootstrap_package_zip.close()

In [13]:
with open("test.zip", "wb") as f:
    bootstrap_package_file.seek(0)
    f.write(bootstrap_package_file.read())

### Upload Bootstrap

In [14]:
dest_url = "https://{0}/on/demandware.servlet/webdav/Sites/Impex/src/instance/{1}.zip".format(
    ENV["server"], dest_file)
webdavsession.put(dest_url, data=bootstrap_package_file)

<Response [201]>

### Activate Bootstrap

In [15]:
response = bmsession.post("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Dispatch".format(ENV["server"]),
                          data={"import" :"", "ImportFileName" : dest_file + ".zip", "realmUse": "False"})

In [16]:
wait_for_import()

### Delete Impex Files

In [17]:
dest_url = "https://{0}/on/demandware.servlet/webdav/Sites/Impex/src/instance/{1}.zip".format(
    ENV["server"], dest_file)
webdavsession.delete(dest_url)

<Response [204]>

## Load Migrations Context

### Export Current Preferences 

In [38]:
# https://dev01-web-stonewall.demandware.net/on/demandware.store/Sites-Site/default/ViewSiteImpex-Dispatch

export_filename = "DWREMigrateExport_{}".format(datetime.now().isoformat())
export_url = "https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Dispatch".format(ENV["server"])
bmsession.post(export_url, data={
        "exportFile": export_filename,
        "export": "",
        "SelectedExportUnit":"GlobalPreferencesExport"
    })


<Response [200]>

In [33]:
wait_for_export(bmsession)

MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

In [ ]:
zipfile = download_zip("https://{}/on/demandware.servlet/webdav/Sites/Impex/src/instance/GlobalMeta.zip".format(ENV["server"]))

## Determine Required Migrations

## Perform Migrations